# Final Project Notebook 2
### Heather Chen
#### 4. Topic modeling using Pyspark

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1607674079580_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Get the names and sizes of these data files using Amazon boto3
import boto3

s3 = boto3.resource('s3')
bucket = 'aws-emr-resources-787469208957-us-east-1'
bucket_resource = s3.Bucket(bucket)
file_sizes = []
file_names = []

for obj in bucket_resource.objects.all():
    if 'gdelt_data' in obj.key and 'bq-results' in obj.key:
        print(obj.last_modified,"\t", round(obj.size * 1e-6), "MB\t",
              obj.key, "\n")
        file_sizes.append(obj.size * 1e-6)
        file_names.append(obj.key)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2020-12-07 14:46:28+00:00 	 4 MB	 gdelt_data/bq-results-0107.csv 

2020-12-07 14:45:53+00:00 	 4 MB	 gdelt_data/bq-results-0108.csv 

2020-12-07 14:47:16+00:00 	 9 MB	 gdelt_data/bq-results-0109.csv 

2020-12-07 14:46:30+00:00 	 6 MB	 gdelt_data/bq-results-0110.csv 

2020-12-07 14:46:04+00:00 	 6 MB	 gdelt_data/bq-results-0111.csv 

2020-12-07 14:49:53+00:00 	 2 MB	 gdelt_data/bq-results-0112.csv 

2020-12-07 14:50:25+00:00 	 7 MB	 gdelt_data/bq-results-0113.csv 

2020-12-07 14:50:13+00:00 	 6 MB	 gdelt_data/bq-results-0114.csv 

2020-12-07 14:50:22+00:00 	 6 MB	 gdelt_data/bq-results-0115.csv 

2020-12-07 14:51:03+00:00 	 10 MB	 gdelt_data/bq-results-0116.csv 

2020-12-07 14:49:21+00:00 	 20 MB	 gdelt_data/bq-results-0117.csv 

2020-12-07 14:50:40+00:00 	 17 MB	 gdelt_data/bq-results-0118.csv 

2020-12-07 14:50:24+00:00 	 10 MB	 gdelt_data/bq-results-0119.csv 

2020-12-07 14:49:22+00:00 	 41 MB	 gdelt_data/bq-results-0120.csv 

2020-12-07 14:49:21+00:00 	 99 MB	 gdelt_data/bq-results-

In [4]:
#Then read these data files to spark dataframe
df_list = []
for filename in file_names:
    df = spark.read.csv('s3://aws-emr-resources-787469208957-us-east-1/' + filename, header=True)
    df_list.append(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#First divide these dataframes to January, Febuary, and March
jan_dfs = df_list[0:25]
feb_dfs = df_list[25:54]
march_dfs = df_list[54:85]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#Given the large volume of our data, we only select China and the U.S to see how their focused topics change over the
#three months
def collect_country_data(df_list, country):
    indx = 0
    for df in df_list:
        df = df.filter(df.CountryCode==country)
        if indx == 0:
            ori_df = df
            indx = indx + 1
            continue
        else:
            curr_df = df
            ori_df = ori_df.union(curr_df)
            indx = indx + 1
    return ori_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#Collect data from China and from US
CH_jan_df = collect_country_data(jan_dfs, 'CH')
CH_feb_df = collect_country_data(feb_dfs, 'CH')
CH_march_df = collect_country_data(march_dfs, 'CH')

US_jan_df = collect_country_data(jan_dfs, 'US')
US_feb_df = collect_country_data(feb_dfs, 'US')
US_march_df = collect_country_data(march_dfs, 'US')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Do basic data cleaning jobs for these dataframes
#(Since our text data is already lowercased, we only do basic regular expressions)
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

def reg_clean_df(df):
    # Remove punctuation (REGEX provided) and numbers
    wrangled = df.withColumn('text', regexp_replace(df.ContextualText, '[_():;,.!?\\-]', ' '))
    wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

    # Merge multiple spaces and drop null values here
    wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))
    wrangled = wrangled.na.drop()
    
    return wrangled

CH_jan_cleaned_df = reg_clean_df(CH_jan_df)
CH_feb_cleaned_df = reg_clean_df(CH_feb_df)
CH_mar_cleaned_df = reg_clean_df(CH_march_df)

US_jan_cleaned_df = reg_clean_df(US_jan_df)
US_feb_cleaned_df = reg_clean_df(US_feb_df)
US_mar_cleaned_df = reg_clean_df(US_march_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Define a function to do the following job before topic modeling
from pyspark.ml.feature import Tokenizer, StopWordsRemover, IDF, CountVectorizer

def pre_analysis(df):
    tokenized_df = Tokenizer(inputCol='text', outputCol='words').transform(df)
    tokenized_df = tokenized_df.na.drop()
    
    # Remove stop words.
    removeStopwords_df = StopWordsRemover(inputCol='words', outputCol='terms').transform(tokenized_df)

    tf_model = CountVectorizer(inputCol="terms", outputCol="features", vocabSize=40000, minDF=5).fit(removeStopwords_df)
    vectorizer = tf_model.transform(removeStopwords_df)
    
    idfizer = IDF(inputCol='features', outputCol='tf_idf_features')
    idf_model = idfizer.fit(vectorizer)
    tfidf_result = idf_model.transform(vectorizer)
    
    #Return a tuple for results and td model
    return tfidf_result, tf_model;

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#Do pre-analysis country by country, month by month
CH_jan_result_df, CH_jan_tf_model = pre_analysis(CH_jan_cleaned_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Show how these tf_idf features look like
CH_jan_result_df.select('tf_idf_features').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|     tf_idf_features|
+--------------------+
|(26826,[0,1,3,19,...|
|(26826,[0,1,4,7,8...|
|(26826,[0,1,4,7,8...|
|(26826,[0,1,4,7,8...|
|(26826,[0,1,2,4,8...|
+--------------------+
only showing top 5 rows

In [12]:
CH_feb_result_df, CH_feb_tf_model = pre_analysis(CH_feb_cleaned_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
CH_mar_result_df, CH_mar_tf_model = pre_analysis(CH_mar_cleaned_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
US_jan_result_df, US_jan_tf_model = pre_analysis(US_jan_cleaned_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
US_feb_result_df, US_feb_tf_model = pre_analysis(US_feb_cleaned_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
US_mar_result_df, US_mar_tf_model = pre_analysis(US_mar_cleaned_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
import pyspark.sql.functions as F
import pyspark.sql.types as T

def get_words(token_list):
    return [vocab[token_id] for token_id in token_list]

#Set the number of topics=5
num_topics = 5
max_iter = 10
num_top_words = 10

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#Do topic modeling and get the topic words 
#China January
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(CH_jan_result_df)
    
vocab = CH_jan_tf_model.vocabulary
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
    
CH_jan_topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
CH_jan_topics.select('topic', 'topicWords').show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------------------------------------------------------------------------------+
|topic|                                                                                  topicWords|
+-----+--------------------------------------------------------------------------------------------+
|    0|     [flight, citizens, said, hubei, kingdom, province, travel, health, british, government]|
|    1|      [cities, million, travel, flights, measures, government, hubei, people, city, beijing]|
|    2|               [cases, human, new, confirmed, market, reported, people, year, sars, chinese]|
|    3|                [case, confirmed, man, cases, health, said, woman, united, people, hospital]|
|    4|[hospital, airport, staff, coronavirus, international, health, new, medical, patients, sars]|
+-----+--------------------------------------------------------------------------------------------+

In [30]:
#China Febuary
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(CH_feb_result_df)
    
vocab = CH_feb_tf_model.vocabulary
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
    
CH_feb_topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
CH_feb_topics.select('topic', 'topicWords').show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------------------------------------------------------------------------+
|topic|                                                                          topicWords|
+-----+------------------------------------------------------------------------------------+
|    0|         [new, said, beijing, people, outbreak, health, u, ship, china, coronavirus]|
|    1|[li, hospital, party, media, patients, january, wuhan, medical, officials, wenliang]|
|    2|      [people, said, cities, city, cases, government, chinese, million, one, flight]|
|    3|       [cases, two, united, u, quarantine, health, citizens, south, said, confirmed]|
|    4| [cases, deaths, new, hubei, confirmed, province, reported, number, mainland, novel]|
+-----+------------------------------------------------------------------------------------+

In [31]:
#China March
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(CH_mar_result_df)
    
vocab = CH_mar_tf_model.vocabulary
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
    
CH_mar_topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
CH_mar_topics.select('topic', 'topicWords').show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------------------------------------------------------------------------------+
|topic|                                                                              topicWords|
+-----+----------------------------------------------------------------------------------------+
|    0|              [people, said, city, virus, cases, beijing, chinese, province, new, hotel]|
|    1|   [city, hubei, people, province, said, restrictions, outbreak, new, lockdown, beijing]|
|    2|[coronavirus, people, us, masks, hospital, said, quarantine, symptoms, beijing, wearing]|
|    3|                [cases, president, chinese, new, xi, virus, trump, first, world, people]|
|    4|    [cases, new, reported, read, hubei, imported, health, deaths, infections, confirmed]|
+-----+----------------------------------------------------------------------------------------+

In [32]:
#U.S. January
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(US_jan_result_df)
    
vocab = US_jan_tf_model.vocabulary
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
    
US_jan_topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
US_jan_topics.select('topic', 'topicWords').show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------------------------------------------------------------------------------------------+
|topic|                                                                                   topicWords|
+-----+---------------------------------------------------------------------------------------------+
|    0|   [plane, base, flight, anchorage, alaska, passengers, nearby, citizens, wednesday, reserve]|
|    1|         [flights, china, cases, airlines, coronavirus, confirmed, white, house, next, trump]|
|    2|                 [case, states, confirmed, reported, first, u, cases, woman, health, chicago]|
|    3|[international, airport, airports, passengers, screening, san, francisco, angeles, los, year]|
|    4|         [county, university, patient, orange, base, department, student, health, local, cdc]|
+-----+---------------------------------------------------------------------------------------------+

In [33]:
#U.S. Febuary
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(US_feb_result_df)
    
vocab = US_feb_tf_model.vocabulary
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
    
US_feb_topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
US_feb_topics.select('topic', 'topicWords').show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------------------------------------------------------------------------------+
|topic|                                                                                 topicWords|
+-----+-------------------------------------------------------------------------------------------+
|    0|          [house, white, trump, billion, county, president, outbreak, funding, health, new]|
|    1|        [base, air, passengers, force, quarantine, san, travis, california, ship, evacuees]|
|    2|        [county, trump, patient, health, officials, case, medical, cdc, school, sacramento]|
|    3|            [county, cases, santa, woman, clara, health, officials, state, case, confirmed]|
|    4|[boston, china, washington, president, trump, massachusetts, coronavirus, said, also, city]|
+-----+-------------------------------------------------------------------------------------------+

In [ ]:
#U.S. March
lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(US_mar_result_df)
    
vocab = US_mar_tf_model.vocabulary
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))
    
US_mar_topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
US_mar_topics.select('topic', 'topicWords').show(truncate=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------------------------------------------------------------------+
|topic|                                                                  topicWords|
+-----+----------------------------------------------------------------------------+
|    0|  [county, cases, confirmed, march, positive, case, covid, health, state, m]|
|    1|[trump, house, white, president, county, package, u, economic, donald, said]|
|    2| [county, people, health, houston, city, order, chicago, said, cases, covid]|
|    3|     [county, cases, ship, new, san, state, washington, cruise, virus, said]|
|    4|      [new, school, seattle, york, said, city, home, march, county, schools]|
+-----+----------------------------------------------------------------------------+